In [3]:
import numpy as np
#import PyKDL as kdl
import math
from typing import Tuple


In [10]:
def rollr(angle: float) -> np.ndarray:
  """Function that returns a 3x3 rotation matrix corresponding to roll

  Args:
      angle (float): angle to rotate by in radians
  Returns:
      np.ndarray: 3x3 rotation matrix 
  """
  R = np.array([[1, 0, 0],
                [0, np.cos(angle), -np.sin(angle)],
                [0, np.sin(angle), np.cos(angle)]
                ])
  return R


def pitchr(angle: float) -> np.ndarray:
  """Function that returns a 3x3 rotation matrix corresponding to pitch

  Args:
      angle (float): angle to rotate by in radians
  Returns:
      np.ndarray: 3x3 rotation matrix 
  """
  R = np.array([[np.cos(angle), 0, np.sin(angle)],
                [0, 1, 0],
                [-np.sin(angle), 0, np.cos(angle)]
                ])
  return R


def yawr(angle: float) -> np.ndarray:
  """Function that returns a 3x3 rotation matrix corresponding to yaw

  Args:
      angle (float): angle to rotate by in radians
  Returns:
      np.ndarray: 3x3 rotation matrix 
  """
  R = np.array([[np.cos(angle), -np.sin(angle), 0],
                [np.sin(angle), np.cos(angle), 0],
                [0, 0, 1]
                ])
  return R


def rpyr(angles: np.ndarray) -> np.ndarray:
  """Do roll pitch yaw for a 3 vector array of angles

  Args:
      angles (np.ndarray): (1,3) vector of angles

  Returns:
      np.ndarray: 3x3 rotaiton matrix
  """
  R = yawr(angles[2]) @ pitchr(angles[1]) @ rollr(angles[0])
  
  return R  


In [5]:
def screw_tf(translation: float, rotation: float, ax: np.ndarray) -> np.ndarray:
  """Create a Transformation Matrix using the angle-axis representation 

  Args:
      translation (float): translation along the axis
      rotation (float): rotation along the axis
      ax (np.ndarray): direction of the axis itself 

  Returns:
      np.ndarray: 4x4 Transformation matrix 
  """
  # Normalize the axis
  a = ax / np.linalg.norm(ax)

  # Precompute cos and sin
  c = np.cos(rotation)
  s = np.sin(rotation)

  # Create the rotation matrix using Equation 2.80 from textbook
  R = np.array([[c + a[0]**2 * (1 - c), a[0]*a[1]*(1 - c) - a[2]*s, a[0]*a[2]*(1 - c) + a[1]*s],
                [a[1]*a[0]*(1 - c) + a[2]*s, c + a[1]**2 * (1 - c), a[1]*a[2]*(1 - c) - a[0]*s],
                [a[2]*a[0]*(1 - c) - a[1]*s, a[2]*a[1]*(1 - c) + a[0]*s, c + a[2]**2 * (1 - c)]
                ])
   
  # Create the transformation matrix
  T = np.eye(4)
  T[0:3, 0:3] = R
  T[0:3, 3] = translation * a

  return T
  


In [6]:
def screw_dh(a: float, alpha: float, d: float, theta: float) -> np.ndarray:
  """Create a frame using the DH Parameters 

  Args:
      a (float): distance from Zi to Zi+1 along the Xi
      alpha (float): angle from z to zi+1 about Xi
      d (float): diestance from Xi-1 to Xi along Zi
      theta (float): angle from Xi-1 to Xi measured about Zi

  Returns:
      np.ndarray: _description_
  """
  # Create the transformation matrix using screw_dh
  Tz = screw_tf(d, theta, np.array([0, 0, 1]))
  Tx = screw_tf(a, alpha, np.array([1, 0, 0]))
  T = Tx @ Tz
  
  return T



In [54]:
def phantom_fk(joint_angles: np.ndarray,
               gimbal_angles: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
  """Create the FK for the phantom and return the full transform and all the homogenous matrices

  Args:
      joint_angles (np.ndarray): a 3x1 matrix of joint angles
      gimbal_angles (np.ndarray): a 3x1 matrix of gimbal angles for the end effector

  Returns: 
      Tuple[np.ndarray, np.ndarray]: 4x4 Full transformation matrix 
                                     All of the 4x4 homogeneous transformation matrices 
  """
  len_1 = 110.0 + 55
  len_2 = 205
  len_3 = 170.0

  # DH Parameters 3 R joints
  T_0_1 = screw_dh(0, 0, 0, joint_angles[0])
  T_1_2 = screw_dh(len_2, 0, 0, joint_angles[1])
  T_2_3 = screw_dh(0, 0, 0, joint_angles[2])
  
  # Wrist gimbal angles
  T_3_e = screw_dh(0, 0, 0, 0)

  # Gimabal rotations
  T_e_g = np.eye(4)
  T_e_g[0:3, 0:3] = rpyr(gimbal_angles)

  # Full Transformation
  phantom_T = np.array([T_0_1, T_1_2, T_2_3, T_3_e, T_e_g], dtype=float)

  phantom_T_0_g = T_0_1 @ T_1_2 @ T_2_3 @ T_3_e @ T_e_g

  return phantom_T_0_g, phantom_T

In [55]:
phantom_fk(np.array([0.,0.,0.]), np.array([0.,0.,0.]))

(array([[  1.,   0.,   0., 205.],
        [  0.,   1.,   0.,   0.],
        [  0.,   0.,   1.,   0.],
        [  0.,   0.,   0.,   1.]]),
 array([[[  1.,   0.,   0.,   0.],
         [  0.,   1.,   0.,   0.],
         [  0.,   0.,   1.,   0.],
         [  0.,   0.,   0.,   1.]],
 
        [[  1.,   0.,   0., 205.],
         [  0.,   1.,   0.,   0.],
         [  0.,   0.,   1.,   0.],
         [  0.,   0.,   0.,   1.]],
 
        [[  1.,   0.,   0.,   0.],
         [  0.,   1.,   0.,   0.],
         [  0.,   0.,   1.,   0.],
         [  0.,   0.,   0.,   1.]],
 
        [[  1.,   0.,   0.,   0.],
         [  0.,   1.,   0.,   0.],
         [  0.,   0.,   1.,   0.],
         [  0.,   0.,   0.,   1.]],
 
        [[  1.,   0.,   0.,   0.],
         [  0.,   1.,   0.,   0.],
         [  0.,   0.,   1.,   0.],
         [  0.,   0.,   0.,   1.]]]))